### Import Libraries

In [ ]:
#For command line use of YOLOv3 (required)
import collections
from collections import deque
from absl import flags
import sys
FLAGS = flags.FLAGS
sys.argv = sys.argv[:1]
FLAGS(sys.argv)

#for writing data to a csv file
import pandas as pd

import time #for calculating FPS
import math
import numpy as np
import cv2 #OpenCV
import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
from tensorflow import keras

#Use the GPU
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

#Check if GPU is being used
print("GPU: ", tf.test.is_gpu_available())

#under yolov3_tf2 folder
from yolov3_tf2.models import YoloV3
from yolov3_tf2.dataset import transform_images #for data augmentation
from yolov3_tf2.utils import convert_boxes, load_darknet_weights, preprocess_image #converts bboxes to deepsort format

#for object tracking
from deep_sort import preprocessing #for max suppressions
from deep_sort import nn_matching #for setting up the association metrics
from deep_sort.detection import Detection #for object detection
from deep_sort.tracker import Tracker #for object tracking information
from tools import generate_detections as gdet #feature generation encoder

# for detecting anomaly in acceleration
from pyod.models.hbos import HBOS
from pyod.models.iforest import IForest
from pyod.models.knn import KNN
from pysad.models.integrations import ReferenceWindowModel
from pysad.utils import ArrayStreamer
from pysad.transform.postprocessing import RunningAveragePostprocessor
from pysad.transform.preprocessing import InstanceUnitNormScaler
from pysad.transform.probability_calibration import GaussianTailProbabilityCalibrator

### Load YOLOv3 Model

In [ ]:
#define classes
class_names = [c.strip() for c in open('./data/labels/coco.names').readlines()]

#define allowed classes:
allowed_classes = ['person', 'bicycle', 'car', 'motorbike', 'bus', 'truck']
vehicle_classes = ['bicycle', 'car', 'motorbike', 'bus', 'truck']

#define paths
weightsyolov3 = './weights/yolov3.weights' # path to weights file
weights= 'checkpoints/yolov3.tf' # path to checkpoints file
size= 416             #resize images to\
checkpoints = 'checkpoints/yolov3.tf'
num_classes = 80      # number of classes in the model

#load model
yolo = YoloV3(classes=num_classes)
load_darknet_weights(yolo, weightsyolov3)
yolo.save_weights(checkpoints)

### Load DeepSORT Tracker

In [ ]:
max_cosine_distance = 0.4 #used to determine if objects between frames are the same
nn_budget = None #used to form a gallery for storing of features
nms_max_overlap = 0.5 #used to avoid too many detections on the same object

model_filename = './model_data/mars-small128.pb' #pretrained CNN for pedestrian tracking
encoder = gdet.create_box_encoder(model_filename, batch_size=8) #feature generations

metric = nn_matching.NearestNeighborDistanceMetric('cosine', max_cosine_distance, nn_budget) #for measuring associations
tracker = Tracker(metric)

### Load Models for Vehicle Acceleration Anomaly Detector

In [ ]:
iterator = ArrayStreamer(shuffle=False) # Streamer to simulate streaming data

In [ ]:
preprocessor = InstanceUnitNormScaler() # Normalizing
postprocessor = RunningAveragePostprocessor(window_size=5) # Running average postprocessor
calibrator = GaussianTailProbabilityCalibrator(running_statistics=True, window_size=6) # Probability calibrator

### Helper Functions

In [ ]:
# function to get intersection of two lines
def line_intersection(line1, line2):
    # Line 1 represented as a1x + b1y = c1
    a1 = line1[1][1] - line1[0][1]
    b1 = line1[0][0] - line1[1][0]
    c1 = a1*(line1[0][0]) + b1*(line1[0][1])
 
    # Line 2 represented as a2x + b2y = c2
    a2 = line2[1][1] - line2[0][1]
    b2 = line2[0][0] - line2[1][0]
    c2 = a2*(line2[0][0]) + b2*(line2[0][1])
 
    determinant = a1*b2 - a2*b1
 
    if (determinant == 0):
        x = 0
        y = 0
    else:
        x = (b2*c1 - b1*c2)/determinant
        y = (a1*c2 - a2*c1)/determinant
    
    return int(x), int(y)

### Track Using Video

In [ ]:
# Uncomment to select video
vid = cv2.VideoCapture('./data/video/Megaworld CCTV/megaworld_accident_1.MP4') #vehicles parallel to one another
# vid = cv2.VideoCapture('./data/video/Megaworld CCTV/megaworld_accident_2.MP4') #camera is too far
# vid = cv2.VideoCapture('./data/video/YouTube Accidents/youtube_accident_1.mp4') #india accident video
# vid = cv2.VideoCapture('./data/video/YouTube Accidents/youtube_accident_2.mp4') #video with the best angle

#give video time to warm up
time.sleep(0.1)

processing_times = deque()

#list for historical trajectory
track_bboxes = [deque(maxlen=30) for _ in range(1000)]
points = [deque(maxlen=30) for _ in range(1000)]
speeds = [deque(maxlen=30) for _ in range(1000)]
accelerations = [deque(maxlen=30) for _ in range(1000)]
acc_anomaly_list = [deque(maxlen=30) for _ in range(1000)]

track_ids = []
overlap_list = []

frame_count = 0

overlap_anomaly = False
acc_anomaly = False
angle_anomaly = True
collision = False

#begin video capturing
while (vid.isOpened()):
    #capture frame
    ret, frame = vid.read()

    if frame is None:
        print('Completed!')
        break

    #if frame is larger than full HD, reduce size to improve performance
    scale = 1280 / max(frame.shape)
    if scale < 1:
        frame = cv2.resize(
            src=frame,
            dsize=None,
            fx=scale,
            fy=scale,
            interpolation=cv2.INTER_AREA
        )
        
    # frame parameters
    frame_height, frame_width = frame.shape[:2]
    
    #preprocessing for YOLOv3 Input
    frame_input = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) #video captured by OpenCV is in BGR format; tensorflow is RGB
    frame_input = tf.expand_dims(frame_input, 0) #expands dims from C,H,W to N,C,H,W
    frame_input = preprocess_image(frame_input, 416) #tensorflow shape is 416
    
    #start the timer
    t1 = time.time() 
    
    #Get detection
    bboxes, scores, classes, nums = yolo(frame_input)
    
    t2 = time.time()
    
    #maximum of 100 bboxes per image
    #boxes: 3D shape (1, 100, 4); 100 max bboxes; 4 = x and y (center coordinates), width, height
    #scores: 2D shape (1, 100); detected objects' confidence scores
    #classes: 2D shape(1, 100); detected objects' classes
    #nums: 1D shape (1); the total number of detected objects
    #these variables are important for DeepSORT
    
    classes = classes[0]
    names = []
    for i in range(len(classes)):
        names.append(class_names[int(classes[i])])
    
    names = np.array(names) #format for Non-Maximum Suppression (NMS)
    numpy_bboxes = np.array(bboxes[0])
    converted_bboxes = convert_boxes(frame, numpy_bboxes) #converts boxes into list
    features = encoder(frame, converted_bboxes) #generate the feature spectra of the detected object
    detections = [Detection(bbox, score, class_name, feature) for bbox, score, class_name, feature 
                  in zip(converted_bboxes, scores[0], names, features)]
    
    #perform non-max suppression to eliminate multiple frames on one target
    boxs = np.array([d.tlwh for d in detections])
    scores = np.array([d.confidence for d in detections])
    classes = np.array([d.class_name for d in detections])
    indices = preprocessing.non_max_suppression(boxs, classes, nms_max_overlap, scores) #indices associate an object with a track
    detections = [detections[i] for i in indices] #removes redundancies
    
    #detections can now be used for DeepSORT since NMS was used to eliminate duplication of the same target
    tracker.predict() #uses Kalman filtering
    tracker.update(detections) #updates the Kalman tracker parameters and filter
    
    cmap = plt.get_cmap('tab20b') #generate color maps
    colors = [cmap(i)[:3] for i in np.linspace(0,1,20)] #generate 20 steps colors

    #show tracked objects
    for track in tracker.tracks:
        if not track.is_confirmed() or track.time_since_update > 1: #if Kalman filtering was not able to assign a track
            continue
        class_name = track.get_class() #get the corresponding classes
        color = colors[int(track.track_id) % len(colors)] #assigning the color code
        color = [i * 255 for i in color] #color originally ranges from 0 to 1; thus must be converted from 0 to 255
        
        if class_name in vehicle_classes:
            bbox = track.to_tlbr() #for OpenCV output minX, minY, maxX, maxY
            cv2.rectangle(frame, (int(bbox[0]), int(bbox[1])), (int(bbox[2]), int(bbox[3])), color, 2) #bounding box rectangle
            cv2.rectangle(frame, (int(bbox[0]), int(bbox[1]-30)), 
                          (int(bbox[0])+(len(class_name)+len(str(track.track_id)))*17, int(bbox[1])), color, -1) #rectangle for text
            cv2.putText(frame, class_name + " - " + str(track.track_id), (int(bbox[0]), int(bbox[1]-10)), 
                        0, 0.75, (255,255,255), 2) #display text for class name and Tracking ID


            center = (int(((bbox[0]) + (bbox[2]))/2), int(((bbox[1]) + (bbox[3]))/2)) #get center coordinates of bounding box
            points[track.track_id].append(center)
            
            # get overlapping bounding boxes
            for track2 in tracker.tracks:
                class2_name = track2.get_class()
                if class2_name in vehicle_classes:
                    bbox2 = track2.to_tlbr()
                    if track.track_id != track2.track_id:
                            # first bbox
                            x1_min = bbox[0]
                            y1_min = bbox[1]
                            x1_max = bbox[2]
                            y1_max = bbox[3]
                            # second bbox
                            x2_min = bbox2[0]
                            y2_min = bbox2[1]
                            x2_max = bbox2[2]
                            y2_max = bbox2[3]
                            # check if the two are not the same detections
                            center1 = (int(((x1_min) + (x1_max))/2), int(((y1_min) + (y1_max))/2)) 
                            center2 = (int(((x2_min) + (x2_max))/2), int(((y2_min) + (y2_max))/2))
                            centroid_distance = math.sqrt( ((center2[0]-center1[0])**2) + ((center2[1]-center1[1])**2) )
                            if centroid_distance > 100:
                                # check for overlaps
                                if (x1_min < x2_max and x2_min < x1_max) and (y1_min < y2_max and y2_min < y1_max):
                                    overlap_anomaly = True
                                    overlap_ids = tuple(sorted((track.track_id, track2.track_id)))
                                    overlap_list.append(overlap_ids)
                                    overlap_list = list(dict.fromkeys(overlap_list))
                                    #print(overlap_list)
                                    break
                                else: 
                                    overlap_anomaly = False
                    
            #compute speed and append to list
            start_point = points[track.track_id][0]
            end_point = points[track.track_id][-1]
            distance = (math.sqrt( ((end_point[0]-start_point[0])**2) + ((end_point[1]-start_point[1])**2) ) / end_point[1]) * 100
            time_period = len(points[track.track_id])
            speed = distance / time_period
            speeds[track.track_id].append(speed)
            cv2.putText(frame, "{:.2f} units/frame".format(speed), (int(bbox[0]), int(bbox[1]-30)), 
                        0, 0.75, (0,255,0), 2) #display text for speed of Tracking ID
            
            #compute acceleration
            if len(speeds[track.track_id]) > 4:
                current_speed = speeds[track.track_id][-1]
                prev_speeds = [speeds[track.track_id][-4], speeds[track.track_id][-3], speeds[track.track_id][-2]]
                avg_prev_speed = sum(prev_speeds) / len(prev_speeds)
                acceleration = current_speed - avg_prev_speed
                accelerations[track.track_id].append(acceleration)
                #print("Accelerations of {}: {}".format(track.track_id, accelerations[track.track_id]))
                
                #detect acceleration anomaly score
                if len(speeds[track.track_id]) == 21:
                    accelerations_reshaped = np.array(accelerations[track.track_id]).reshape(-1, 1)
                    model = ReferenceWindowModel(model_cls=HBOS, window_size=20, sliding_size=10, initial_window_X=accelerations_reshaped) # model initialization
                elif len(speeds[track.track_id]) > 22:
                    if acceleration != 0:
                        acc_processed = preprocessor.fit_transform_partial([acceleration])
                        anomaly_score = model.fit_score_partial(acc_processed)
                        anomaly_score = postprocessor.fit_transform_partial(anomaly_score)
                        calibrated_score = calibrator.fit_transform_partial(anomaly_score)
                        #if calibrated_score > 0.95:
                        cv2.putText(frame, "{:.2f}".format(calibrated_score), (int(bbox[0]), int(bbox[3]+30)), 
                                    0, 0.75, (0,255,0), 2)
                        acc_anomaly_list[track.track_id].append(calibrated_score)
            
            # get projected direction
            if len(points[track.track_id]) > 4:
                prev_x_points = [points[track.track_id][-4][0], points[track.track_id][-3][0], points[track.track_id][-2][0]]
                prev_y_points = [points[track.track_id][-4][1], points[track.track_id][-3][1], points[track.track_id][-2][1]]
                avg_x_points = sum(prev_x_points) / len(prev_x_points)
                avg_y_points = sum(prev_y_points) / len(prev_y_points)
                prev_point = (int(avg_x_points), int(avg_y_points))
                delta_y = (end_point[1] - prev_point[1]) 
                delta_x = (end_point[0] - prev_point[0])
                cv2.arrowedLine(frame, (end_point), (end_point[0]+(delta_x*10),end_point[1]+(delta_y*10)), (0,255,0), 2)
            
            # begin collision detection on overlapping objects
            if  len(overlap_list)>0:
                for overlap_objs in overlap_list:
                    obj1 = overlap_objs[0]
                    obj2 = overlap_objs[1]
                    if len(points[obj1]) > 4 and len(points[obj2]) > 4:
                        # get line coordinates of obj1
                        obj1_curr_point = tuple(points[obj1][-1])
                        obj1_prev_x_points = [points[obj1][-4][0], points[obj1][-3][0], points[obj1][-2][0]]
                        obj1_prev_y_points = [points[obj1][-4][1], points[obj1][-3][1], points[obj1][-2][1]]
                        obj1_avg_x_points = sum(obj1_prev_x_points) / len(obj1_prev_x_points)
                        obj1_avg_y_points = sum(obj1_prev_y_points) / len(obj1_prev_y_points)
                        obj1_prev_point = (int(obj1_avg_x_points), int(obj1_avg_y_points))
                        obj1_delta_y = (obj1_curr_point[1] - obj1_prev_point[1]) 
                        obj1_delta_x = (obj1_curr_point[0] - obj1_prev_point[0])
                        obj1_next_point = (obj1_curr_point[0]+(obj1_delta_x*10), obj1_curr_point[1]+(obj1_delta_y*10))
                        line1 = [obj1_curr_point, obj1_next_point]
                        # get line coordinates of obj2
                        obj2_curr_point = tuple(points[obj2][-1])
                        obj2_prev_x_points = [points[obj2][-4][0], points[obj2][-3][0], points[obj2][-2][0]]
                        obj2_prev_y_points = [points[obj2][-4][1], points[obj2][-3][1], points[obj2][-2][1]]
                        obj2_avg_x_points = sum(obj2_prev_x_points) / len(obj2_prev_x_points)
                        obj2_avg_y_points = sum(obj2_prev_y_points) / len(obj2_prev_y_points)
                        obj2_prev_point = (int(obj2_avg_x_points), int(obj2_avg_y_points))
                        obj2_delta_y = (obj2_curr_point[1] - obj2_prev_point[1]) 
                        obj2_delta_x = (obj2_curr_point[0] - obj2_prev_point[0])
                        obj2_next_point = (obj2_curr_point[0]+(obj2_delta_x*10), obj2_curr_point[1]+(obj2_delta_y*10))
                        line2 = [obj2_curr_point, obj2_next_point]
                        # get intersection of the two lines
                        intersection = line_intersection(line1,line2)
                        if intersection[0] != None and intersection[1] != None:
                            cv2.circle(frame, (intersection[0],intersection[1]), 10, (0,0,255), -1) #visualize intersection
                            
                            if len(speeds[obj1]) > 22 and len(speeds[obj2]) > 22: #minimum number of speed values for anomaly detection
                                if acc_anomaly_list[obj1][-1] > 0.9 and acc_anomaly_list[obj2][-1] > 0.9: #threshold of anomaly score
                                    cv2.putText(frame, "Collision Detected!", (30, frame_height-30), 0, 1, (0,0,255), 2)
            
            #for historical trajectory
            for j in range(1, len(points[track.track_id])):
                if points[track.track_id][j-1] is None or points[track.track_id][j] is None: #check if current and previous tracker has a center point
                    continue
                thickness = int(np.sqrt(64/float(j+1))*2) #closer points are visually thinner
                cv2.line(frame, (points[track.track_id][j-1]), (points[track.track_id][j]), color, thickness)

    processing_times.append(t1 - t2)
    #use processing times from last 200 frames
    if len(processing_times) > 200:
        processing_times.popleft()

    _, f_width = frame.shape[:2]
    # get mean processing time [ms]
    processing_time = np.mean(processing_times) * 1000
    #get fps
    fps = 1000 / processing_time
    #display fps
    cv2.putText(
        frame,
        f"Inference Time: {processing_time:.1f}ms ({fps:.1f} FPS)",
        (20, 40),
        cv2.FONT_HERSHEY_COMPLEX,
        f_width / 1000,
        (0,0,255),
        1,
        cv2.LINE_AA
    )

    frame_count += 1

    #display date and time
    current_time = time.asctime( time.localtime(time.time()) )
    cv2.putText(frame, current_time, (0,80), 0, 1, (255,255,255), 2)

    #display video stream
    cv2.imshow('Video', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'): #press Q to quit
        break

#clear stream capture
vid.release()
cv2.destroyAllWindows()
cv2.waitKey(1)